In [9]:
from data import get_data
from neural_network import NeuralNetwork
from randomforest.randomForestInterface import RandomForest
import numpy as np

In [2]:
train_X, train_y, val_X, val_y, test_X, test_y = get_data()

In [6]:
models = [NeuralNetwork(), RandomForest()]
predictions = []
for model in models:
  model.train(train_X, train_y, val_X, val_y)
  predictions.append(model.predict(test_X))

Finished training! Time elapsed: 169.44280695915222


In [10]:
for i, prediction in enumerate(predictions):
  print(f"Model {i} has an accuracy of {np.mean(prediction == test_y)}")

Model 0 has an accuracy of 0.863849765258216
Model 1 has an accuracy of 0.8497652582159625
